# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Лабораторная работа 4

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [4]:
import pandas as pd
df = pd.read_csv("data_reviews_sample.csv")
print(df)

import pandas as pd
rf = pd.read_csv('data_recipes_sample.csv')
print(rf)

        Unnamed: 0     user_id  recipe_id        date  rating  \
0           370476       21752      57993  2003-05-01       5   
1           624300      431813     142201  2007-09-16       5   
2           187037      400708     252013  2008-01-10       4   
3           706134  2001852463     404716  2017-12-11       5   
4           312179       95810     129396  2008-03-14       5   
...            ...         ...        ...         ...     ...   
126691     1013457     1270706     335534  2009-05-17       4   
126692      158736     2282344       8701  2012-06-03       0   
126693     1059834      689540     222001  2008-04-08       5   
126694      453285  2000242659     354979  2015-06-02       5   
126695      691207      463435     415599  2010-09-30       5   

                                                   review  
0       Last week whole sides of frozen salmon fillet ...  
1       So simple and so tasty!  I used a yellow capsi...  
2       Very nice breakfast HH, easy to

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [27]:
import pandas
import random

n = 30000 #кол-во строк в файле
s = 1500 #желаемое кол-во строк: 30000*0.05
filename = "data_recipes_sample.csv"
skip = sorted(random.sample(range(n),n-s))
dff = pandas.read_csv(filename, skiprows=skip)

n1 = 126696
s1 = 6334 #126696*0.05
filename1 = "data_reviews_sample.csv"
skip1 = sorted(random.sample(range(n1),n1-s1))
dff1 = pandas.read_csv(filename1, skiprows=skip1)

In [28]:
with pd.ExcelWriter('recipes.xlsx') as writer:  
    dff.to_excel(writer, sheet_name='Рецепты')
    dff1.to_excel(writer, sheet_name='Отзывы')
print(dff,dff1)

                blepandekager   danish   apple pancakes  503475   50  128473  \
0                               pizza pan  potato skins   60938   60   35635   
1                       secret ingredient  bbq meatloaf  342620   75   50969   
2                     windy s  sweet and sour meatballs  276594   50  341338   
3                mcgriddle  pancakes breakfast sandwich  134085   15   31364   
4                                rainbow chicken strips  192542   50  360369   
...                                                 ...     ...  ...     ...   
1495                          zucchini friendship bread  322225   75  736108   
1496                                zucchini rice pilaf  282349   25  705251   
1497  zucchini  corn  black bean  and jack cheese qu...   36349   30    3288   
1498               zwetschgenkuchen  bavarian plum cake  386977  240  177443   
1499             cookies by design   cookies on a stick  298512   29  506822   

      2013-07-08  10.0  \
0     2003-04

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [10]:
import xlwings as xw
from datetime import datetime, timedelta

# открыть книгу Excel
wb = xw.Book('recipes.xlsx')

# выбрать лист Рецепты
sheet = wb.sheets['Рецепты']

# добавить столбец seconds_assign
sheet.range('D9').value = 'seconds_assign'

# выбрать столбцы с временем начала и окончания рецепта
start_time_range = sheet.range('B1:B100')
end_time_range = sheet.range('C1:C100')

# вычислить время выполнения рецепта в секундах
values = ['D9:D100']
import datetime
str(values.datetime(seconds=666))

# присвоить массив значений диапазону ячеек столбца seconds_assign
sheet.range('D9:D100').value = values


AttributeError: 'list' object has no attribute 'datetime'

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
last_column = sheet.range(1, sheet.cells.shape[1]).end('left').column
new_column = last_column + 1
sheet.range(1, new_column).value = 'seconds_formula'
sheet.range(2, new_column).formula = '=TIMEVALUE("00:"&B2)*86400'
last_row = sheet.range(sheet.cells.shape[0], new_column).end('up').row
sheet.range(2, new_column).expand('down').formula = '=TIMEVALUE("00:"&B2)*86400'

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
header_range = sheet.range('D9:D100')
header_range.font.bold = True
header_range.horizontal_alignment = xw.constants.HAlign.center

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
def validate(reviews, recipes):
    for review in reviews:
        rating = review.get("rating")
        recipe_id = review.get("recipe_id")
        recipe_exists = False
        
        # Check if rating is between 0 and 5
        if rating is None or not isinstance(rating, int) or rating < 0 or rating > 5:
            # Highlight the review in red
            print(f"\033[91m{review}\033[0m")
            continue
        
        # Check if recipe_id exists in recipes
        for recipe in recipes:
            if recipe.get("recipe_id") == recipe_id:
                recipe_exists = True
                break
        
        if not recipe_exists:
            # Highlight the review in red
            print(f"\033[91m{review}\033[0m")


9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 